# Obtención de datos

En este código se obtiene una segunda lista de datos de interés

In [ ]:
# paquetes
import neurokit2 as nk # incluye herramientas de análisis y detección de ondas de ECG
import pandas as pd # para trabajar con DataFrames
import numpy as np # para manejar tipos de números
import math # para operaciones especiales
import statistics # para operaciones especiales
import os # para acciones del sistema operativo

In [ ]:
# creamos una función que nos regrese la información principal de cada ECG
def analiza(id_ecg, categoria, senal, sr=100):
    
    # usamos un try-except para las señales conflictivas
    try:
        
        # primero analizamos la información
        senal_limpia = nk.ecg_clean(senal, sampling_rate=sr, method='neurokit') # quitamos el ruido de la señal
        aux_rc, picos_R = nk.ecg_peaks(senal_limpia, sampling_rate=sr) # ubicamos los picos de las ondas R
        _, picos_otros = nk.ecg_delineate(senal_limpia, picos_R, sampling_rate=sr, method="peak") # ubicamos los otros picos

        # segundo calculamos los tiempos entre ondas
        ritmo_cardiaco = nk.ecg_rate(aux_rc, sampling_rate=sr)
        tiempos_PQ = [(y-x)/sr for x,y in zip(picos_otros['ECG_P_Peaks'],picos_otros['ECG_Q_Peaks'])]
        tiempos_QR = [(y-x)/sr for x,y in zip(picos_otros['ECG_Q_Peaks'],picos_R['ECG_R_Peaks'])]
        tiempos_RS = [(y-x)/sr for x,y in zip(picos_R['ECG_R_Peaks'],picos_otros['ECG_S_Peaks'])]
        tiempos_ST = [(y-x)/sr for x,y in zip(picos_otros['ECG_S_Peaks'],picos_otros['ECG_T_Peaks'])]
        tiempos_TP = [(y-x)/sr for x,y in zip(picos_otros['ECG_T_Peaks'][:-1],picos_otros['ECG_P_Peaks'][1:])]

        #tercero quitamos nan's
        ritmo_cardiaco = [x for x in ritmo_cardiaco if not math.isnan(x)]
        tiempos_PQ = [x for x in tiempos_PQ if not math.isnan(x)]
        tiempos_QR = [x for x in tiempos_QR if not math.isnan(x)]
        tiempos_RS = [x for x in tiempos_RS if not math.isnan(x)]
        tiempos_ST = [x for x in tiempos_ST if not math.isnan(x)]
        tiempos_TP = [x for x in tiempos_TP if not math.isnan(x)]
        
        # cuarto calculamos estadisticos de distancias
        min_RC = min(ritmo_cardiaco)                # los valores RC están en latidos por minuto (lpm).
        media_RC = statistics.mean(ritmo_cardiaco)
        max_RC = max(ritmo_cardiaco)             
        sd_RC = statistics.stdev(ritmo_cardiaco)
        min_PQ = min(tiempos_PQ)                    # los valores de aquí para abajo están en segundos.
        media_PQ = statistics.mean(tiempos_PQ)
        max_PQ = max(tiempos_PQ)
        sd_PQ = statistics.stdev(tiempos_PQ)
        min_QR = min(tiempos_QR)
        media_QR = statistics.mean(tiempos_QR)
        max_QR = max(tiempos_QR)
        sd_QR = statistics.stdev(tiempos_QR)
        min_RS = min(tiempos_RS)
        media_RS = statistics.mean(tiempos_RS)
        max_RS = max(tiempos_RS)
        sd_RS = statistics.stdev(tiempos_RS)
        min_ST = min(tiempos_ST)
        media_ST = statistics.mean(tiempos_ST)
        max_ST = max(tiempos_ST)
        sd_ST = statistics.stdev(tiempos_ST)
        min_TP = min(tiempos_TP)
        media_TP = statistics.mean(tiempos_TP)
        max_TP = max(tiempos_TP)
        sd_TP = statistics.stdev(tiempos_TP)
        res = [id_ecg, categoria, 
               min_RC, media_RC, max_RC, sd_RC,
               min_PQ, media_PQ, max_PQ, sd_PQ,
               min_QR, media_QR, max_QR, sd_QR,
               min_RS, media_RS, max_RS, sd_RS,
               min_ST, media_ST, max_ST, sd_ST,
               min_TP, media_TP, max_TP, sd_TP]

    except:
        
        # en caso de no poder realizar el proceso, regresa una lista de NaN's
        res = [None for x in range(24)]
        res = [id_ecg, categoria] + res
        print("except en ecg " + id_ecg)
    return res

In [ ]:
# creamos una función que explore los archivos .csv de la carpeta con ECG
# y nos regrese un resumen de cada ECG como un renglón de un Data Frame
def explorar(direccion, tipo = ".csv"):

    # creamos el dataframe ya con los títulos
    df = pd.DataFrame(columns = ['id_ecg', 'categoria', 
                                 'min_RC', 'media_RC', 'max_RC', 'sd_RC',
                                 'min_PQ', 'media_PQ', 'max_PQ', 'sd_PQ',
                                 'min_QR', 'media_QR', 'max_QR', 'sd_QR',
                                 'min_RS', 'media_RS', 'max_RS', 'sd_RS',
                                 'min_ST', 'media_ST', 'max_ST', 'sd_ST',
                                 'min_TP', 'media_TP', 'max_TP', 'sd_TP'])
    i = 0 # para saber en que renglón vamos
    for root, dirs, files in os.walk(direccion): # os.walk va a ingresar a cada carpeta de la dirección y regresar el nombre de los archivos dentro
        for name in files: # para cada archivo...
            if name.endswith(tipo): # si el archivo es .csv...
                id_ecg = name[8:-4] # el nombre es 'patient_####.csv' entonces hacemos slicing para quitar "patient_" y ".csv".
                cat = root[13:] # el root es 'Datos_Leonel\carpeta' y queremos solo la carpeta, hacemos slicing.
                archivo = root + "\\" + name # la dirección del archivo se obtiene concatenando root y file_name
                senales = pd.read_csv(archivo, index_col='Unnamed: 0') # las señales están guardadas en formato .csv 
                senal_II = senales['II'] # Queremos la señal DII
                renglon = analiza(id_ecg=id_ecg, categoria=cat, senal=senal_II, sr=100) # aplicamos 'analiza' a DII
                df.loc[i,:] = renglon # concatena el resultado de 'analiza' al final del df
                i += 1 # contabilizamos el renglón
    return df # retornamos el dataframe

In [ ]:
# ejecutamos y guardamos el df para acceder a él más fácilmente
# solo como seguridad (para evitar repetir el proceso en vano)
# agregamos una llave que debe ajustarse a True si se quiere volver a calcular el dataframe

recalcular_df = True # llave
nombre_archivo = 'df_datos_tiempos.csv'

if recalcular_df:
    df1 = explorar(direccion='Datos_Leonel') # hacemos el df
    df1 = df1.astype({'id_ecg':float}) # ajustamos dtype
    df2 = pd.read_csv('ptbxl_database.csv') # traemos metadata
    df1 = df1.merge(df2[['ecg_id','patient_id','age','sex','height','weight']], 
                      how='left', left_on='id_ecg',right_on='ecg_id',sort=True) # juntamos la data
    df1.drop(labels='ecg_id', axis = 1, inplace=True) # eliminamos columna repetida
    df1.to_csv(nombre_archivo) # guardamos
    print("datos analizados y guardados :)")